In [1]:
import pandas as pd
import numpy as np
raw_df = pd.read_csv('TIMESERIES_all_monthly_enriched_no_dairyntel.csv').ffill()

In [2]:
def remove_invalid_features(df, min_num_valid=100):
    v_check = df.ffill().isna().sum() < len(df) - min_num_valid
    v_feats = v_check[v_check].index.tolist()
#     if 'date' in v_feats:
#         v_feats.remove('date')
#     if 'target' in v_feats:
#         v_feats.remove('target')
    return [f for f in v_feats if len(df[f].unique()) > 1]

In [3]:
# target_var = "ZuivelNL - Dutch butter quotation"
input_df = raw_df.copy() 
target_var = "ZuivelNL - Dutch SMP food quotation"
time_horizon = 8
input_df['Target'] = -np.sign(input_df[target_var].diff(-time_horizon))
input_df['date'] = pd.to_datetime(raw_df.apply(lambda x: x['date'][:10],axis=1))

In [4]:
valid_feats = remove_invalid_features(input_df)
data = input_df[valid_feats].dropna()
data['Target'] = data['Target'].astype('int')

In [ ]:
import pycaret
from pycaret.classification import *
params = {
    'target': 'Target',
    'train_size':0.8,
    'normalize' : True,
    'silent':True,
    'feature_selection':True,
    'feature_selection_threshold':0.1,
}
exp_clf = setup(data, **params)
compare_models()

 
Setup Succesfully Completed!


,Description,Value
0,session_id,3094
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(93, 1814)"
4,Missing Values,False
5,Numeric Features,1748
6,Categorical Features,64
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


IntProgress(value=0, description='Processing: ', max=170)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:38:47
Status,. . . . . . . . . . . . . . . . . .,Fitting Fold 5 of 10
Estimator,. . . . . . . . . . . . . . . . . .,CatBoost Classifier
ETC,. . . . . . . . . . . . . . . . . .,9.09 Minutes Remaining


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,SVM - Linear Kernel,0.9607,0.0000,0.9167,1.0000,0.9514,0.9196
1,Ridge Classifier,0.9607,0.0000,0.9417,0.9800,0.9546,0.9196
2,Logistic Regression,0.9357,0.9917,0.9167,0.9600,0.9292,0.8696
3,Light Gradient Boosting Machine,0.9357,0.9875,0.9167,0.9600,0.9292,0.8696
4,Extra Trees Classifier,0.8964,0.9750,0.9000,0.9100,0.8988,0.7940
5,Extreme Gradient Boosting,0.8839,0.9854,0.8667,0.9217,0.8763,0.7666
6,K Neighbors Classifier,0.8714,0.9646,0.9250,0.8583,0.8814,0.7440
7,Random Forest Classifier,0.8696,0.9365,0.8167,0.9300,0.8570,0.7361
8,Ada Boost Classifier,0.8696,0.9750,0.9167,0.8750,0.8807,0.7309
9,Linear Discriminant Analysis,0.8696,0.9396,0.7917,0.9333,0.8352,0.7387


In [ ]:
model = tune_model('rf' ,n_iter=100)

In [ ]:
# from solar.backtesting import Backtesting
# bt_clf = Backtesting(data, model, time_horizon, valid_feats, task='classification')

In [ ]:
model